In [8]:
import pandas as pd
import numpy as np

# Fund Administrator Report
admin_data = {
    'ISIN': ['US0378331005', 'US5949181045', 'IT0005433195', 'US02079K3059', 'DE0007664039'],
    'AssetName': ['Apple Inc.', 'Microsoft Corp.', 'BTP Italia 2.5% 2030', 'Alphabet Inc.', 'Volkswagen AG'],
    'Quantity': [50000, 10000, 50000, 20000, 30000],
    'Price': [200.00, 400.00, 100.00, 150.00, 160.00],
    'AssetClass': ['Equity', 'Equity', 'Gov Bond', 'Equity', 'Equity']
}
df_external = pd.DataFrame(admin_data)
df_external['MarketValue'] = df_external['Quantity'] * df_external['Price']

# Internal Report
internal_data = {
    'ISIN': ['US0378331005', 'US5949181045', 'US02079K3059', 'DE0007664039', 'FR0000120271'],
    'AssetName': ['Apple Inc.', 'Microsoft Corp.', 'Alphabet Inc.', 'Volkswagen AG', 'LVMH'],
    'Quantity': [50000, 9800, 20000, 30000, 10000],
    'Price': [199.90, 400.00, 149.25, 160.00, 750.00],
    'AssetClass': ['Equity', 'Equity', 'Equity', 'Equity', 'Equity']
}
df_internal = pd.DataFrame(internal_data)
df_internal['MarketValue'] = df_internal['Quantity'] * df_internal['Price']

print("DataFrame df_external e df_internal creati.")
print(f"Admin Portfolio Total Value: {df_external['MarketValue'].sum():,.2f} €")
print(f"Internal Portfolio Total Value: {df_internal['MarketValue'].sum():,.2f} €")
print(f"Total DIfference (internal-admin): {df_internal['MarketValue'].sum()-df_external['MarketValue'].sum():,.2f} €")

DataFrame df_external e df_internal creati.
Admin Portfolio Total Value: 26,800,000.00 €
Internal Portfolio Total Value: 29,200,000.00 €
Total DIfference (internal-admin): 2,400,000.00 €
None


In [9]:
def reconcile_portfolios(df_internal, df_external):
    """Reconcile two portfolio reports to find differences
        
    Args:
        df_internal (pd.DataFrame): Internal System DataFrame. 
                                    Must contain ['ISIN', 'Quantity', 'Price'].
        df_external (pd.DataFrame): External System DataFrame.
                                    Must contain ['ISIN', 'Quantity', 'Price'].

    Returns:
        pd.DataFrame: Dataframe that summarizes the differences classified per type, 
    """

    # merge with outer to keep both dataframes values and information
    df_total = pd.merge(df_internal, df_external, on='ISIN', how='outer', indicator=True)
    # replace nan with 0
    numeric_cols = df_total.select_dtypes(include='number').columns
    df_total[numeric_cols] = df_total[numeric_cols].fillna(value=0.)

    # change names, calculate differences and drop extra columns
    df_total['AssetName'] = df_total['AssetName_x'].fillna(df_total['AssetName_y'])
    df_total['AssetClass'] = df_total['AssetClass_x'].fillna(df_total['AssetClass_y'])
    df_total = df_total.rename(columns={'MarketValue_x': 'MarketValue_internal', 'MarketValue_y': 'MarketValue_external', 'Price_x': 'Price_internal',  'Price_y': 'Price_external', 'Quantity_x': 'Quantity_internal', 'Quantity_y': 'Quantity_external'})
    df_total['diff_quantity'] = df_total['Quantity_internal']-df_total['Quantity_external']
    df_total['diff_price'] = df_total['Price_internal']-df_total['Price_external']
    df_infos = df_total.drop(columns = ['AssetName_x', 'AssetName_y', 'AssetClass_x', 'AssetClass_y'])

    # build conditions 
    conditions = [(df_infos['_merge']=='right_only'), (df_infos['_merge']=='left_only'), (df_infos['diff_quantity']!=0), (df_infos['diff_price']!=0)]
    message = ['Missing in internal', 'Extra in internal', 'Quantity Mismatch', 'Price Mismatch']
    df_infos['break_type'] =  np.select(conditions, message, default='OK')

    return df_infos[df_infos['break_type']!='OK'].loc[:, ['ISIN', 'AssetName', 'MarketValue_internal', 'MarketValue_external', 'break_type']]
    # print(df_infos)

In [7]:
print(reconcile_portfolios(df_internal, df_external))

           ISIN             AssetName  MarketValue_internal  \
1  FR0000120271                  LVMH             7500000.0   
2  IT0005433195  BTP Italia 2.5% 2030                   0.0   
3  US02079K3059         Alphabet Inc.             2985000.0   
4  US0378331005            Apple Inc.             9995000.0   
5  US5949181045       Microsoft Corp.             3920000.0   

   MarketValue_Admin           break_type  
1                0.0  Missing in internal  
2          5000000.0    Extra in internal  
3          3000000.0       Price Mismatch  
4         10000000.0       Price Mismatch  
5          4000000.0    Quantity Mismatch  
